In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Add
from keras.models import Model
from keras.optimizers import SGD
import keras.backend as K
import os
K.clear_session()
print("Running Tensorflow version", tf.__version__)
print("Keras is running on", K.backend(), "backend")

Using TensorFlow backend.


Running Tensorflow version 2.1.0
Keras is running on tensorflow backend


In [2]:
# problem with model with rounding 
'''
def roundingAlgo(x): 
    # first one that works with model_1 & model_2 
    # problem - this rounding function is slow: model_2 = 3 hours / epoch
    # comparison, model_0 = 20 mins / epoch
    # in addition, off by half with integer inputs (lower than actual value, e.g. floor(2) ≈ 1.5, floor(2.01) ≈ 2)
    # source: https://en.wikipedia.org/wiki/Floor_and_ceiling_functions#Continuity_and_series_expansions
    if True:
        result = x - 0.5
        for p in range(1, 7):
            result = result + K.sin(x * p * 2 * math.pi) / (p * math.pi)
    return result
# '''
'''     
def roundingAlgo(x):
    # second one that works with model_2 
    # problem - this rounding function is slower than first working algo: model_2 = 4,2 hours / epoch
    # comparison, model_0 = 20 mins / epoch
    # source: self
    return x - x % 1
# '''
# '''
def roundingAlgo(x): 
    # simplification of the first algo loop by simplifying the expression for range(1,7)
    # problem - rounding function is still slow = 2,5 hours / epoch
    # all non-speed problem of first algo still applies
    result = x - 0.5
    resultCos = K.cos(2 * math.pi * x)
    return result + K.sin(2 * math.pi * x) * (1 + resultCos) * (13 + 2 * resultCos - 18 * K.pow(resultCos, 2) - 32 * K.pow(resultCos, 3) + 80 * K.pow(resultCos, 4)) / 15
# '''
'''
def roundingAlgo(x): 
    # made to fool the engine to have a gradient
    return 0 * x + K.round(x)
# '''


# check https://github.com/keras-team/keras/issues/2218
# check https://github.com/keras-team/keras/issues/2221
# https://www.tensorflow.org/api_docs/python/tf/custom_gradient
class RoundClampQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundClampQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundClampQ7_12, self).build(input_shape)
    def call(self, X):
        return K.clip(roundingAlgo(X * 4096), -524288, 524287) / 4096.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundClampQ7_12, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class RoundOverflowQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_12, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 4096) + 524288) % 1048576) - 524288) / 4096.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundOverflowQ7_12, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class RoundClampQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundClampQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundClampQ3_4, self).build(input_shape)
    def call(self, X):
        return K.clip(roundingAlgo(X * 16), -128, 127) / 16.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundClampQ3_4, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class RoundOverflowQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ3_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 128) % 256) - 128) / 16.0
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(RoundOverflowQ3_4, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
# TODO: change IdentityFinalLayer into the final layer specified in https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg
# Reference: https://github.com/BoXiao123/py-tiny-yolo-from-scratch/blob/master/main.py
# Reference: https://github.com/BoXiao123/py-tiny-yolo-from-scratch/blob/master/forward_region.py
# Reference: https://blog.paperspace.com/how-to-implement-a-yolo-v3-object-detector-from-scratch-in-pytorch-part-2/
class IdentityFinalLayer(Layer):
    def __init__(self, **kwargs):
        super(IdentityFinalLayer, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        config = {"name": self.__class__.__name__}
        base_config = super(IdentityFinalLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
def DBL(previousLayer, layerFilter, kernelSize=(3, 3), roundingFunction=Identity()):
    return roundingFunction(LeakyReLU()(roundingFunction(BatchNormalization()(Conv2D(filters=layerFilter, kernel_size=kernelSize, padding='same')(previousLayer)))))
print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
from pycocotools.coco import COCO
nms = 0
coco = 0
''' // comment on this line to enable/disable this block
# this block uses COCO annotation to find how many classification for learning (in case of updates)
dataDir='.'
dataType='train2017'
coco=COCO('{}/annotations/instances_{}.json'.format(dataDir,dataType))
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
# '''
if isinstance(nms, list):
    classificationClass = len(nms)
else:
    classificationClass = 80

print("Number of class classification is", classificationClass)
''' // comment on this line to enable/disable this block
# this block runs a sample of target data generation
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math
import os

imgIds = coco.getImgIds(catIds=coco.getCatIds(catNms=['person','dog','skateboard']));
img = coco.loadImgs(imgIds[int(len(imgIds) / 2)])[0]

%matplotlib inline
imgLoad = Image.open('{}/{}/{}'.format(dataDir, dataType, img['file_name']))
imgWRatio = 448 / imgLoad.size[0]
imgHRatio = 448 / imgLoad.size[1]
imgLoad = np.asarray(imgLoad.resize((448,448)))

print("Sample image input")

imgplot = plt.imshow(imgLoad)
plt.show()

annotations = coco.loadAnns(ids=coco.getAnnIds(imgIds=int(img['file_name'].split('.')[0])))
bbox_array = list(map(lambda x: x['bbox'], annotations))
cat_array = list(map(lambda x: x['category_id'], annotations))
bigBox_array = []
smolBox_array = []
toAppendArray = []
for x in range(len(bbox_array)):
    print("object", x)
    print(x, "-", bbox_array[x], "\tcategory", cat_array[x])
    toAppendArray = bbox_array[x]
    toAppendArray[0] = math.floor(toAppendArray[0] * imgWRatio * 100) / 100
    toAppendArray[1] = math.floor(toAppendArray[1] * imgHRatio * 100) / 100
    toAppendArray[2] = math.floor(toAppendArray[2] * imgWRatio * 100) / 100
    toAppendArray[3] = math.floor(toAppendArray[3] * imgHRatio * 100) / 100
    if bbox_array[x][2] * bbox_array[x][3] > 10000:
        if len(bigBox_array) < 3:
            bigBox_array.append([cat_array[x], toAppendArray])
    else:
        if len(smolBox_array) < 3:
            smolBox_array.append([cat_array[x], toAppendArray])
# https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
# https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch
# https://medium.com/oracledevs/final-layers-and-loss-functions-of-single-stage-detectors-part-1-4abbfa9aa71c
print('Big Box')
for p in range(len(bigBox_array)):
    print(bigBox_array[p])
print('Small Box')
for p in range(len(smolBox_array)):
    print(smolBox_array[p])


# bigBox get small grid
smallGrid = np.zeros(shape=(14, 14, 3 * (5 + classificationClass)), dtype=np.float32)
for x in bigBox_array:
    for p in range(3):
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass)] = 1
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 1] = x[1][0]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 2] = x[1][1]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 3] = x[1][2]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 4] = x[1][3]
        smallGrid[math.floor(x[1][0] / 32), math.floor(x[1][1] / 32), p * (5 + classificationClass) + 4 + x[0]] = 1
# smolBox get big grid
bigGrid = np.zeros(shape=(28, 28, 3 * (5 + classificationClass)), dtype=np.float32)
for x in smolBox_array:
    for p in range(3):
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass)] = 1
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 1] = x[1][0]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 2] = x[1][1]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 3] = x[1][2]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 4] = x[1][3]
        bigGrid[math.floor(x[1][0] / 16), math.floor(x[1][1] / 16), p * (5 + classificationClass) + 4 + x[0]] = 1
        
print(K.constant(value=smallGrid, dtype='float32', shape=(14, 14, 3 * (classificationClass + 5))).shape)
print(K.constant(value=bigGrid, dtype='float32', shape=(28, 28, 3 * (classificationClass + 5))).shape)
i = 0
with (os.scandir('{}/{}/'.format(dataDir, dataType))) as trainingPictures:
    for currentPicture in trainingPictures:
        print(currentPicture.name)
        i = i + 1
        if i > 9:
            break
# '''

del coco

Number of class classification is 80


In [4]:
print()
''' // comment on this line to enable/disable this block
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import math
import os
# '''

# added quick-fix for categories that are not included in COCO dataset (2017) - https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
def cocoCategoryQF(value):
    subtractValue = 0
    if value > 12:
        subtractValue = subtractValue + 1
        if value > 26:
            subtractValue = subtractValue + 1
            if value > 30:
                subtractValue = subtractValue + 2
                if value > 45:
                    subtractValue = subtractValue + 1
                    if value > 66:
                        subtractValue = subtractValue + 1
                        if value > 69:
                            subtractValue = subtractValue + 2
                            if value > 71:
                                subtractValue = subtractValue + 1
                                if value > 83:
                                    subtractValue = subtractValue + 1
                                    if value > 91:
                                        subtractValue = subtractValue + 1
    return value - subtractValue
# todo: fix problem with target data generation - accuracy is unacceptable
def getImageParam_iter(dataDir, dataType, coco=None):
    if coco == None:
        coco = COCO('{}/annotations/instances_{}.json'.format(dataDir,dataType))
    with (os.scandir('{}/{}/'.format(dataDir, dataType))) as trainingPictures:
        for currentPicture in trainingPictures:
            if currentPicture.is_file():
                imgLoad = Image.open('{}/{}/{}'.format(dataDir, dataType, currentPicture.name)).convert("RGB")
                imgWRatio = 448 / imgLoad.size[0]
                imgHRatio = 448 / imgLoad.size[1]
                imgLoad = np.asarray(imgLoad.resize((448,448)))
                annotations = coco.loadAnns(ids=coco.getAnnIds(imgIds=int(currentPicture.name.split('.')[0])))
                bbox_array = list(map(lambda x: x['bbox'], annotations))
                cat_array = list(map(lambda x: x['category_id'], annotations))
                bigBox_array = []
                smolBox_array = []
                toAppendArray = []
                for x in range(len(bbox_array)):
                    toAppendArray = bbox_array[x]
                    toAppendArray[0] = math.floor(toAppendArray[0] * imgWRatio)# * 100) / 100
                    toAppendArray[1] = math.floor(toAppendArray[1] * imgHRatio)# * 100) / 100
                    toAppendArray[2] = math.floor(toAppendArray[2] * imgWRatio)# * 100) / 100
                    toAppendArray[3] = math.floor(toAppendArray[3] * imgHRatio)# * 100) / 100
                    if bbox_array[x][2] * bbox_array[x][3] > 10000:
                        if len(bigBox_array) < 3:
                            bigBox_array.append([cocoCategoryQF(cat_array[x]), toAppendArray])
                    else:
                        if len(smolBox_array) < 3:
                                smolBox_array.append([cocoCategoryQF(cat_array[x]), toAppendArray])
                smallGrid = np.zeros(shape=(14, 14, 3 * (5 + classificationClass)), dtype=np.float32)
                for x in bigBox_array:
                    for p in range(3):
                        pos1 = math.floor(x[1][0] / 32)
                        pos2 = math.floor(x[1][1] / 32)
                        if pos1 > 13:
                            pos1 = 13
                        elif pos1 < 0:
                            pos1 = 0
                        if pos2 > 13:
                            pos2 = 13
                        elif pos2 < 0:
                            pos2 = 0
                        smallGrid[pos1, pos2, p * (5 + classificationClass)] = 1.0
                        smallGrid[pos1, pos2, p * (5 + classificationClass) + 1] = x[1][0] / 32.0
                        smallGrid[pos1, pos2, p * (5 + classificationClass) + 2] = x[1][1] / 32.0
                        smallGrid[pos1, pos2, p * (5 + classificationClass) + 3] = x[1][2] / 32.0
                        smallGrid[pos1, pos2, p * (5 + classificationClass) + 4] = x[1][3] / 32.0
                        smallGrid[pos1, pos2, p * (5 + classificationClass) + 4 + x[0]] = 1.0
                # smolBox get big grid
                bigGrid = np.zeros(shape=(28, 28, 3 * (5 + classificationClass)), dtype=np.float32)
                for x in smolBox_array:
                    for p in range(3):
                        pos1 = math.floor(x[1][0] / 16)
                        pos2 = math.floor(x[1][1] / 16)
                        if pos1 > 27:
                            pos1 = 27
                        elif pos1 < 0:
                            pos1 = 0
                        if pos2 > 27:
                            pos2 = 27
                        elif pos2 < 0:
                            pos2 = 0
                        bigGrid[pos1, pos2, p * (5 + classificationClass)] = 1.0
                        bigGrid[pos1, pos2, p * (5 + classificationClass) + 1] = x[1][0] / 16.0
                        bigGrid[pos1, pos2, p * (5 + classificationClass) + 2] = x[1][1] / 16.0
                        bigGrid[pos1, pos2, p * (5 + classificationClass) + 3] = x[1][2] / 16.0
                        bigGrid[pos1, pos2, p * (5 + classificationClass) + 4] = x[1][3] / 16.0
                        bigGrid[pos1, pos2, p * (5 + classificationClass) + 4 + x[0]] = 1.0
                yield (np.expand_dims(imgLoad, axis=0), [np.expand_dims(bigGrid, axis=0), np.expand_dims(smallGrid, axis=0)])
print("Target data generator successfully defined")


Target data generator successfully defined


In [5]:
# model_0 does no rounding (float32 operation)
model_0_input = Input(shape=(448, 448, 3))
model_0_pointer = model_0_input
print("Input shape:", model_0_pointer.shape) # 448 x 448 x 3
model_0_startBranch = DBL(previousLayer=model_0_input, layerFilter=16) 
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 448 x 448 x 16
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 224 x 224 x 16
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=32)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 224 x 224 x 32
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 112 x 112 x 32
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=64)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 112 x 112 x 64
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 56 x 56 x 64
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=128)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 56 x 56 x 128
model_0_startBranch = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 28 x 28 x 128
model_0_startBranch = DBL(previousLayer=model_0_startBranch, layerFilter=256)
model_0_pointer = model_0_startBranch
print(model_0_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_0_branch0
model_0_branch0 = MaxPooling2D(pool_size=(2, 2))(model_0_startBranch)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 256
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=512)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_0_branch0)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=1024)
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 1024
model_0_branch0 = DBL(previousLayer=model_0_branch0, layerFilter=256, kernelSize=(1, 1))
model_0_pointer = model_0_branch0
print(model_0_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_0_branch0 (14 x 14 x 256), following model_0_branch00
model_0_branch00 = DBL(previousLayer=model_0_branch0, layerFilter=128, kernelSize=(1, 1))
model_0_pointer = model_0_branch00
print(model_0_pointer.shape) # 14 x 14 x 128
model_0_branch00 = UpSampling2D()(model_0_branch00)
model_0_pointer = model_0_branch00
print(model_0_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_0_branch1 (unchanged from model_0_startBranch) and model_0_branch00
model_0_mergedBranch = Concatenate()([model_0_startBranch, model_0_branch00])
model_0_pointer = model_0_mergedBranch
print(model_0_pointer.shape) # 28 x 28 x 384
model_0_mergedBranch = DBL(previousLayer=model_0_mergedBranch, layerFilter=256)
model_0_pointer = model_0_mergedBranch
print(model_0_pointer.shape) # 28 x 28 x 256
model_0_mergedBranch = DBL(previousLayer=model_0_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_0_pointer = model_0_mergedBranch
print("Model output 0 shape:", model_0_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_0_mergedBranch = IdentityFinalLayer()(model_0_mergedBranch)
print() # OUTPUT = model_0_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_0_branch01
model_0_branch01 = DBL(previousLayer=model_0_branch0, layerFilter=512)
model_0_pointer = model_0_branch01
print(model_0_pointer.shape) # 14 x 14 x 512
model_0_branch01 = DBL(previousLayer=model_0_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_0_pointer = model_0_branch01
print("Model output 1 shape:", model_0_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_0_branch01 = IdentityFinalLayer()(model_0_branch01)
print() # OUTPUT = model_0_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_0 = Model(inputs=model_0_input, outputs=[model_0_mergedBranch, model_0_branch01])
model_0.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_0 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 255)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 255)

Model model_0 compilation complete


In [6]:
# model_1 approximates Q7.12 signed fixed point operations with floating point rules (overflow = maximum/minimum value) 
# Done by rounding to the nearest 1/4096 and capping at [-128, 128) after batch normalization and activation layers
model_1_input = Input(shape=(448, 448, 3))
model_1_pointer = model_1_input
print("Input shape:", model_1_pointer.shape) # 448 x 448 x 3
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_input, layerFilter=16) 
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 448 x 448 x 16
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 224 x 224 x 16
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=32)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 224 x 224 x 32
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 112 x 112 x 32
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=64)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 112 x 112 x 64
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 56 x 56 x 64
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=128)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 56 x 56 x 128
model_1_startBranch = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 28 x 28 x 128
model_1_startBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_startBranch, layerFilter=256)
model_1_pointer = model_1_startBranch
print(model_1_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_1_branch0
model_1_branch0 = MaxPooling2D(pool_size=(2, 2))(model_1_startBranch)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 256
model_1_branch0 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=512)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_1_branch0)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch0 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=1024)
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 1024
model_1_branch0 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=256, kernelSize=(1, 1))
model_1_pointer = model_1_branch0
print(model_1_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_1_branch0 (14 x 14 x 256), following model_1_branch00
model_1_branch00 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=128, kernelSize=(1, 1))
model_1_pointer = model_1_branch00
print(model_1_pointer.shape) # 14 x 14 x 128
model_1_branch00 = UpSampling2D()(model_1_branch00)
model_1_pointer = model_1_branch00
print(model_1_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_1_branch1 (unchanged from model_1_startBranch) and model_1_branch00
model_1_mergedBranch = Concatenate()([model_1_startBranch, model_1_branch00])
model_1_pointer = model_1_mergedBranch
print(model_1_pointer.shape) # 28 x 28 x 384
model_1_mergedBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_mergedBranch, layerFilter=256)
model_1_pointer = model_1_mergedBranch
print(model_1_pointer.shape) # 28 x 28 x 256
model_1_mergedBranch = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_1_pointer = model_1_mergedBranch
print("Model output 0 shape:", model_1_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_1_mergedBranch = IdentityFinalLayer()(model_1_mergedBranch)
print() # OUTPUT = model_1_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1") # following model_1_branch01
model_1_branch01 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch0, layerFilter=512)
model_1_pointer = model_1_branch01
print(model_1_pointer.shape) # 14 x 14 x 512
model_1_branch01 = DBL(roundingFunction=RoundClampQ7_12(), previousLayer=model_1_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_1_pointer = model_1_branch01
print("Model output 1 shape:", model_1_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_1_branch01 = IdentityFinalLayer()(model_1_branch01)
print() # OUTPUT = model_1_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_1 = Model(inputs=model_1_input, outputs=[model_1_mergedBranch, model_1_branch01])
model_1.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_1 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 255)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 255)

Model model_1 compilation complete


In [7]:
# model_2 approximates Q7.12 signed fixed point operations with integer rules (overflow = positive -> negative & vice versa) 
# Done by rounding to the nearest 1/4096 and capping at [-128, 128) after batch normalization and activation layers
model_2_input = Input(shape=(448, 448, 3))
model_2_pointer = model_2_input
print("Input shape:", model_2_pointer.shape) # 448 x 448 x 3
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_input, layerFilter=16) 
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 448 x 448 x 16
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 224 x 224 x 16
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=32)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 224 x 224 x 32
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 112 x 112 x 32
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=64)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 112 x 112 x 64
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 56 x 56 x 64
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=128)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 56 x 56 x 128
model_2_startBranch = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 28 x 28 x 128
model_2_startBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_startBranch, layerFilter=256)
model_2_pointer = model_2_startBranch
print(model_2_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_2_branch0
model_2_branch0 = MaxPooling2D(pool_size=(2, 2))(model_2_startBranch)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 256
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=512)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_2_branch0)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=1024)
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 1024
model_2_branch0 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=256, kernelSize=(1, 1))
model_2_pointer = model_2_branch0
print(model_2_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_2_branch0 (14 x 14 x 256), following model_2_branch00
model_2_branch00 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=128, kernelSize=(1, 1))
model_2_pointer = model_2_branch00
print(model_2_pointer.shape) # 14 x 14 x 128
model_2_branch00 = UpSampling2D()(model_2_branch00)
model_2_pointer = model_2_branch00
print(model_2_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_2_branch1 (unchanged from model_2_startBranch) and model_2_branch00
model_2_mergedBranch = Concatenate()([model_2_startBranch, model_2_branch00])
model_2_pointer = model_2_mergedBranch
print(model_2_pointer.shape) # 28 x 28 x 384
model_2_mergedBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_mergedBranch, layerFilter=256)
model_2_pointer = model_2_mergedBranch
print(model_2_pointer.shape) # 28 x 28 x 256
model_2_mergedBranch = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_2_pointer = model_2_mergedBranch
print("Model output 0 shape:", model_2_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_2_mergedBranch = IdentityFinalLayer()(model_2_mergedBranch)
print() # OUTPUT = model_2_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_2_branch01
model_2_branch01 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch0, layerFilter=512)
model_2_pointer = model_2_branch01
print(model_2_pointer.shape) # 14 x 14 x 512
model_2_branch01 = DBL(roundingFunction=RoundOverflowQ7_12(), previousLayer=model_2_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_2_pointer = model_2_branch01
print("Model output 1 shape:", model_2_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_2_branch01 = IdentityFinalLayer()(model_2_branch01)
print() # OUTPUT = model_2_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_2 = Model(inputs=model_2_input, outputs=[model_2_mergedBranch, model_2_branch01])
model_2.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_2 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 255)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 255)

Model model_2 compilation complete


In [8]:
# model_3 approximates Q3.4 signed fixed point operations with floating point rules (overflow = maximum/minimum value) 
# Done by rounding to the nearest 1/16 and capping at [-8, 8) after batch normalization and activation layers
model_3_input = Input(shape=(448, 448, 3))
model_3_pointer = model_3_input
print("Input shape:", model_3_pointer.shape) # 448 x 448 x 3
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_input, layerFilter=16) 
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 448 x 448 x 16
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 224 x 224 x 16
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=32)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 224 x 224 x 32
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 112 x 112 x 32
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=64)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 112 x 112 x 64
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 56 x 56 x 64
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=128)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 56 x 56 x 128
model_3_startBranch = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 28 x 28 x 128
model_3_startBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_startBranch, layerFilter=256)
model_3_pointer = model_3_startBranch
print(model_3_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_3_branch0
model_3_branch0 = MaxPooling2D(pool_size=(2, 2))(model_3_startBranch)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 256
model_3_branch0 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=512)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_3_branch0)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch0 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=1024)
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 1024
model_3_branch0 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=256, kernelSize=(1, 1))
model_3_pointer = model_3_branch0
print(model_3_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_3_branch0 (14 x 14 x 256), following model_3_branch00
model_3_branch00 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=128, kernelSize=(1, 1))
model_3_pointer = model_3_branch00
print(model_3_pointer.shape) # 14 x 14 x 128
model_3_branch00 = UpSampling2D()(model_3_branch00)
model_3_pointer = model_3_branch00
print(model_3_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_3_branch1 (unchanged from model_3_startBranch) and model_3_branch00
model_3_mergedBranch = Concatenate()([model_3_startBranch, model_3_branch00])
model_3_pointer = model_3_mergedBranch
print(model_3_pointer.shape) # 28 x 28 x 384
model_3_mergedBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_mergedBranch, layerFilter=256)
model_3_pointer = model_3_mergedBranch
print(model_3_pointer.shape) # 28 x 28 x 256
model_3_mergedBranch = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_3_pointer = model_3_mergedBranch
print("Model output 0 shape:", model_3_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_3_mergedBranch = IdentityFinalLayer()(model_3_mergedBranch)
print() # OUTPUT = model_3_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1") # following model_3_branch01
model_3_branch01 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch0, layerFilter=512)
model_3_pointer = model_3_branch01
print(model_3_pointer.shape) # 14 x 14 x 512
model_3_branch01 = DBL(roundingFunction=RoundClampQ3_4(), previousLayer=model_3_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_3_pointer = model_3_branch01
print("Model output 1 shape:", model_3_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_3_branch01 = IdentityFinalLayer()(model_3_branch01)
print() # OUTPUT = model_3_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_3 = Model(inputs=model_3_input, outputs=[model_3_mergedBranch, model_3_branch01])
model_3.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_3 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 255)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 255)

Model model_3 compilation complete


In [9]:
# model_4 approximates Q3.4 signed fixed point operations with integer rules (overflow = positive -> negative & vice versa) 
# Done by rounding to the nearest 1/16 and capping at [-8, 8) after batch normalization and activation layers
model_4_input = Input(shape=(448, 448, 3))
model_4_pointer = model_4_input
print("Input shape:", model_4_pointer.shape) # 448 x 448 x 3
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_input, layerFilter=16) 
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 448 x 448 x 16
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 224 x 224 x 16
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=32)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 224 x 224 x 32
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 112 x 112 x 32
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=64)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 112 x 112 x 64
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 56 x 56 x 64
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=128)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 56 x 56 x 128
model_4_startBranch = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 28 x 28 x 128
model_4_startBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_startBranch, layerFilter=256)
model_4_pointer = model_4_startBranch
print(model_4_pointer.shape) # 28 x 28 x 256
print("Branch split from main branch - following branch 0") # 2 branch split from startBranch (28 x 28 x 256), following model_4_branch0
model_4_branch0 = MaxPooling2D(pool_size=(2, 2))(model_4_startBranch)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 256
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=512)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch0 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(model_4_branch0)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=1024)
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 1024
model_4_branch0 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=256, kernelSize=(1, 1))
model_4_pointer = model_4_branch0
print(model_4_pointer.shape) # 14 x 14 x 256
print("Branch split from branch 0 - following branch 0,0") # 2 branch split from model_4_branch0 (14 x 14 x 256), following model_4_branch00
model_4_branch00 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=128, kernelSize=(1, 1))
model_4_pointer = model_4_branch00
print(model_4_pointer.shape) # 14 x 14 x 128
model_4_branch00 = UpSampling2D()(model_4_branch00)
model_4_pointer = model_4_branch00
print(model_4_pointer.shape) # 28 x 28 x 128
print("Branch merge from branch 1 and branch 0,0") # 2 branch merge from model_4_branch1 (unchanged from model_4_startBranch) and model_4_branch00
model_4_mergedBranch = Concatenate()([model_4_startBranch, model_4_branch00])
model_4_pointer = model_4_mergedBranch
print(model_4_pointer.shape) # 28 x 28 x 384
model_4_mergedBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_mergedBranch, layerFilter=256)
model_4_pointer = model_4_mergedBranch
print(model_4_pointer.shape) # 28 x 28 x 256
model_4_mergedBranch = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_mergedBranch, layerFilter=3 * (4 + 1 + classificationClass))
model_4_pointer = model_4_mergedBranch
print("Model output 0 shape:", model_4_pointer.shape) # 28 x 28 x (3 * (5 + classificationClass))
model_4_mergedBranch = IdentityFinalLayer()(model_4_mergedBranch)
print() # OUTPUT = model_4_mergedBranch (note: 26 x 26 grid untuk deteksi objek kecil)

print("Branch split from branch 0 - following branch 0,1")# following model_4_branch01
model_4_branch01 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch0, layerFilter=512)
model_4_pointer = model_4_branch01
print(model_4_pointer.shape) # 14 x 14 x 512
model_4_branch01 = DBL(roundingFunction=RoundOverflowQ3_4(), previousLayer=model_4_branch01, layerFilter=3 * (4 + 1 + classificationClass))
model_4_pointer = model_4_branch01
print("Model output 1 shape:", model_4_pointer.shape) # 14 x 14 x (3 * (5 + classificationClass))
model_4_branch01 = IdentityFinalLayer()(model_4_branch01)
print() # OUTPUT = model_4_branch01 (note: 13 x 13 grid untuk deteksi objek besar)

model_4 = Model(inputs=model_4_input, outputs=[model_4_mergedBranch, model_4_branch01])
model_4.compile(optimizer=SGD(lr=0.001, decay=0.0005, momentum=0.9), loss='mean_absolute_error', metrics=['accuracy'])
print("Model model_4 compilation complete")

Input shape: (None, 448, 448, 3)
(None, 448, 448, 16)
(None, 224, 224, 16)
(None, 224, 224, 32)
(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 128)
(None, 28, 28, 128)
(None, 28, 28, 256)
Branch split from main branch - following branch 0
(None, 14, 14, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 1024)
(None, 14, 14, 256)
Branch split from branch 0 - following branch 0,0
(None, 14, 14, 128)
(None, 28, 28, 128)
Branch merge from branch 1 and branch 0,0
(None, 28, 28, 384)
(None, 28, 28, 256)
Model output 0 shape: (None, 28, 28, 255)

Branch split from branch 0 - following branch 0,1
(None, 14, 14, 512)
Model output 1 shape: (None, 14, 14, 255)

Model model_4 compilation complete


In [10]:
print(model_0.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 448, 448, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 448, 448, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
identity_1 (Identity)           multiple             0           batch_normalization_1[0][0]      
                                                                 leaky_re_lu_1[0][0]        

In [11]:
print(model_1.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 448, 448, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 448, 448, 16) 64          conv2d_14[0][0]                  
__________________________________________________________________________________________________
round_clamp_q7_12_1 (RoundClamp (None, 448, 448, 16) 0           batch_normalization_14[0][0]     
                                                                 leaky_re_lu_14[0][0]       

In [12]:
print(model_2.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 448, 448, 16) 448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_27 (BatchNo (None, 448, 448, 16) 64          conv2d_27[0][0]                  
__________________________________________________________________________________________________
round_overflow_q7_12_1 (RoundOv (None, 448, 448, 16) 0           batch_normalization_27[0][0]     
                                                                 leaky_re_lu_27[0][0]       

In [13]:
print(model_3.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 448, 448, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_40 (BatchNo (None, 448, 448, 16) 64          conv2d_40[0][0]                  
__________________________________________________________________________________________________
round_clamp_q3_4_1 (RoundClampQ (None, 448, 448, 16) 0           batch_normalization_40[0][0]     
                                                                 leaky_re_lu_40[0][0]       

In [14]:
print(model_4.summary())

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 448, 448, 16) 448         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 448, 448, 16) 64          conv2d_53[0][0]                  
__________________________________________________________________________________________________
round_overflow_q3_4_1 (RoundOve (None, 448, 448, 16) 0           batch_normalization_53[0][0]     
                                                                 leaky_re_lu_53[0][0]       

In [15]:
# setup
epoch_steps = 39000
epoch_number = 3
val_steps = 2500
val_freq = [1,3]
cocoTrain = COCO('{}/annotations/instances_{}.json'.format('.', 'train2017'))
cocoVal = COCO('{}/annotations/instances_{}.json'.format('.', 'val2017'))
if epoch_steps * epoch_number > 39429 * 3:
    epoch_steps = 39429
    epoch_number = 3
if isinstance(val_freq, list):
    if val_steps * len(val_freq) > 5000:
        val_steps = 5000 / epoch_number
        val_freq = 1
else:
    if val_steps * epoch / val_freq > 5000:
        val_steps = 5000 / epoch_number
        val_freq = 1

loading annotations into memory...
Done (t=21.92s)
creating index...
index created!
loading annotations into memory...
Done (t=0.79s)
creating index...
index created!


In [16]:
print()
# ''' // comment on this line to enable/disable this block
model_0_history = model_0.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()


Epoch 1/3
39000/39000 [==============================] - 2349s 60ms/step - loss: 0.3351 - identity_final_layer_1_loss: 0.1510 - identity_final_layer_2_loss: 0.1841 - identity_final_layer_1_accuracy: 0.0046 - identity_final_layer_2_accuracy: 9.6337e-04 - val_loss: 0.0194 - val_identity_final_layer_1_loss: 0.3767 - val_identity_final_layer_2_loss: 0.3689 - val_identity_final_layer_1_accuracy: 0.0016 - val_identity_final_layer_2_accuracy: 3.8571e-04
Epoch 2/3
39000/39000 [==============================] - 2371s 61ms/step - loss: 0.2116 - identity_final_layer_1_loss: 0.0938 - identity_final_layer_2_loss: 0.1179 - identity_final_layer_1_accuracy: 0.0020 - identity_final_layer_2_accuracy: 0.0015 - val_loss: 0.0194 - val_identity_final_layer_1_loss: 0.3767 - val_identity_final_layer_2_loss: 0.3689 - val_identity_final_layer_1_accuracy: 0.0016 - val_identity_final_layer_2_accuracy: 3.8571e-04
Epoch 3/3
39000/39000 [==============================] - 2921s 75ms/step - loss: 0.1831 - identity_fi

In [17]:
print()
# ''' // comment on this line to enable/disable this block
model_1_history = model_1.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()


Epoch 1/3
39000/39000 [==============================] - 3769s 97ms/step - loss: 0.7944 - identity_final_layer_3_loss: 0.3690 - identity_final_layer_4_loss: 0.4255 - identity_final_layer_3_accuracy: 8.0328e-04 - identity_final_layer_4_accuracy: 8.3883e-04 - val_loss: 0.6520 - val_identity_final_layer_3_loss: 0.4232 - val_identity_final_layer_4_loss: 0.2300 - val_identity_final_layer_3_accuracy: 0.0000e+00 - val_identity_final_layer_4_accuracy: 0.0000e+00
Epoch 2/3
39000/39000 [==============================] - 3704s 95ms/step - loss: 0.6194 - identity_final_layer_3_loss: 0.2590 - identity_final_layer_4_loss: 0.3604 - identity_final_layer_3_accuracy: 6.9434e-05 - identity_final_layer_4_accuracy: 0.0000e+00 - val_loss: 0.6520 - val_identity_final_layer_3_loss: 0.4232 - val_identity_final_layer_4_loss: 0.2300 - val_identity_final_layer_3_accuracy: 0.0000e+00 - val_identity_final_layer_4_accuracy: 0.0000e+00
Epoch 3/3
39000/39000 [==============================] - 3774s 97ms/step - loss: 

In [18]:
print()
# ''' // comment on this line to enable/disable this block
model_2_history = model_2.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()


Epoch 1/3
39000/39000 [==============================] - 7328s 188ms/step - loss: nan - identity_final_layer_5_loss: nan - identity_final_layer_6_loss: nan - identity_final_layer_5_accuracy: 0.9974 - identity_final_layer_6_accuracy: 0.9932 - val_loss: nan - val_identity_final_layer_5_loss: nan - val_identity_final_layer_6_loss: nan - val_identity_final_layer_5_accuracy: 0.9976 - val_identity_final_layer_6_accuracy: 0.9931
Epoch 2/3
39000/39000 [==============================] - 7006s 180ms/step - loss: nan - identity_final_layer_5_loss: nan - identity_final_layer_6_loss: nan - identity_final_layer_5_accuracy: 0.9974 - identity_final_layer_6_accuracy: 0.9933 - val_loss: nan - val_identity_final_layer_5_loss: nan - val_identity_final_layer_6_loss: nan - val_identity_final_layer_5_accuracy: 0.9976 - val_identity_final_layer_6_accuracy: 0.9931
Epoch 3/3
39000/39000 [==============================] - 7299s 187ms/step - loss: nan - identity_final_layer_5_loss: nan - identity_final_layer_6_l

In [19]:
print()
# ''' // comment on this line to enable/disable this block
model_3_history = model_3.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()


Epoch 1/3
39000/39000 [==============================] - 3738s 96ms/step - loss: 0.8793 - identity_final_layer_7_loss: 0.4312 - identity_final_layer_8_loss: 0.4481 - identity_final_layer_7_accuracy: 0.0036 - identity_final_layer_8_accuracy: 3.6761e-05 - val_loss: 0.7694 - val_identity_final_layer_7_loss: 0.4701 - val_identity_final_layer_8_loss: 0.3002 - val_identity_final_layer_7_accuracy: 0.0000e+00 - val_identity_final_layer_8_accuracy: 0.0000e+00: 0.8874 - identity_ - ETA: 8:21 - loss: 0.8871 - identity_final_layer_7_lo - ETA: 7s - loss: 0.8794 - identity_final_layer_7_loss: 0.4312 - identity_final_layer_8_loss: 0.4482 - 
Epoch 2/3
39000/39000 [==============================] - 3602s 92ms/step - loss: 0.8028 - identity_final_layer_7_loss: 0.3787 - identity_final_layer_8_loss: 0.4242 - identity_final_layer_7_accuracy: 0.0000e+00 - identity_final_layer_8_accuracy: 0.0000e+00 - val_loss: 0.7694 - val_identity_final_layer_7_loss: 0.4701 - val_identity_final_layer_8_loss: 0.3002 - val_

In [20]:
print()
# ''' // comment on this line to enable/disable this block
model_4_history = model_4.fit_generator(
    generator=getImageParam_iter('.', 'train2017', cocoTrain), 
    steps_per_epoch=epoch_steps, 
    epochs=epoch_number,
    validation_data=getImageParam_iter('.', 'val2017', cocoVal),
    validation_steps=val_steps,
    validation_freq=val_freq
)
# '''
print()


Epoch 1/3
39000/39000 [==============================] - 7321s 188ms/step - loss: nan - identity_final_layer_9_loss: nan - identity_final_layer_10_loss: nan - identity_final_layer_9_accuracy: 0.9972 - identity_final_layer_10_accuracy: 0.9955 - val_loss: nan - val_identity_final_layer_9_loss: nan - val_identity_final_layer_10_loss: nan - val_identity_final_layer_9_accuracy: 0.9974 - val_identity_final_layer_10_accuracy: 0.9954_layer_9_loss: nan - identity_final_layer_10_loss: nan - identity_final_layer_9_accuracy: 0.9969 - identity_final_layer_10_ - ETA: 1:46:44 - loss: nan - identity_final_layer_9_loss: nan - identity_final_l - ETA: 1:46:25 - loss: nan - identity_final_layer_9_loss: nan - identity_final_layer_10 - ETA: 1:29:16 - loss: nan - identity_final_layer_9_loss: nan - identity_fi - ETA: 1:28:57 - loss: nan - identity_final_layer_9_loss: nan - identity_final_layer_10_loss: nan - identity_fi - ETA: 1:28:45 - loss: nan - identity_final_layer_9_loss: nan - identity_final_layer_10_l

39000/39000 [==============================] - 7332s 188ms/step - loss: nan - identity_final_layer_9_loss: nan - identity_final_layer_10_loss: nan - identity_final_layer_9_accuracy: 0.9973 - identity_final_layer_10_accuracy: 0.9955 - val_loss: nan - val_identity_final_layer_9_loss: nan - val_identity_final_layer_10_loss: nan - val_identity_final_layer_9_accuracy: 0.9974 - val_identity_final_layer_10_accuracy: 0.9954



In [21]:
del cocoTrain
del cocoVal
print("Resource successfully released")

Resource successfully released
